In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms, utils, datasets
from tqdm import tqdm
from torch.nn.parameter import Parameter
import torchvision
import os
from IPython.core.ultratb import AutoFormattedTB
__ITB__ = AutoFormattedTB(mode = 'Verbose',color_scheme='LightBg', tb_offset = 1)

In [11]:
class BevDataset(Dataset):
  def __init__(self, root, size=512, train=True):
    postfix = 'train' if train else 'test'
    root = os.path.join(root, 'bev_classification', 'images')
    self.dataset_folder = torchvision.datasets.ImageFolder(os.path.join(root, postfix) ,transform = transforms.Compose([transforms.Resize((size,size)),transforms.ToTensor()]))
    # print(self.dataset_folder.class_to_idx)


  def __getitem__(self,index): # TODO: Update the label
    img = self.dataset_folder[index]
    return img[0], img[1]
  
  def __len__(self):
    return len(self.dataset_folder)

In [12]:
# Hyper parameters
batch_size = 50

In [13]:
train_data = DataLoader(BevDataset('.'), batch_size=batch_size, shuffle=True)
count = 0
for x, y_truth in train_data:
    print(x.shape, y_truth)
    break


(tensor([[[0.0196, 0.0196, 0.0157,  ..., 0.4588, 0.4549, 0.4549],
         [0.0196, 0.0196, 0.0157,  ..., 0.4588, 0.4549, 0.4549],
         [0.0196, 0.0196, 0.0157,  ..., 0.4588, 0.4588, 0.4588],
         ...,
         [0.0353, 0.0353, 0.0353,  ..., 0.8745, 0.8706, 0.8706],
         [0.0392, 0.0392, 0.0392,  ..., 0.8667, 0.8627, 0.8627],
         [0.0392, 0.0392, 0.0392,  ..., 0.8627, 0.8588, 0.8588]],

        [[0.0980, 0.0980, 0.0941,  ..., 0.4196, 0.4157, 0.4157],
         [0.0980, 0.0980, 0.0941,  ..., 0.4196, 0.4157, 0.4157],
         [0.0980, 0.0980, 0.0941,  ..., 0.4196, 0.4196, 0.4196],
         ...,
         [0.2118, 0.2118, 0.2118,  ..., 0.4000, 0.4039, 0.4039],
         [0.2157, 0.2157, 0.2157,  ..., 0.3922, 0.3961, 0.3961],
         [0.2157, 0.2157, 0.2157,  ..., 0.3882, 0.3922, 0.3922]],

        [[0.0549, 0.0549, 0.0510,  ..., 0.4118, 0.4078, 0.4078],
         [0.0549, 0.0549, 0.0510,  ..., 0.4118, 0.4078, 0.4078],
         [0.0549, 0.0549, 0.0510,  ..., 0.4118, 0.4118, 0